In [1]:
import pandas as pd
import numpy as np
import datetime
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

2024-07-19 18:13:12.188701: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
price_df = pd.read_csv('data/training/in/input/filtered_price.csv')
news_df = pd.read_csv('data/training/in/input/price_news_return_embedding.csv')

In [3]:
news_df

,Date,Ticker,Open,High,Low,Close,Volume,Latest_News,Concatenated_News,1_day_return,2_day_return,3_day_return,4_day_return,7_day_return,10_day_return,30_day_return,Concatenated_News_Embedding,Latest_News_Embedding
0,2020-05-29,CSCO,41.665731,43.276611,41.095586,43.276611,43553600.0,Stocks to Buy in a Post-Pandemic World,Stocks to Buy in a Post-Pandemic World - US In...,-0.032829,-0.019833,-0.018747,-0.020487,0.004787,-0.061016,-0.033012,"[0.001997282262891531, -0.018077492713928223, ...","[0.011239121668040752, -0.02143305167555809, -..."
1,2020-05-29,JWN,15.714695,15.826675,14.930826,15.052138,25072100.0,"Nordstrom (JWN) Q1 Loss Wider Than Expected, S...","Nordstrom (JWN) Q1 Loss Wider Than Expected, S...",0.027728,0.092801,0.169840,0.243788,0.240226,0.085082,-0.050814,"[-0.032860055565834045, -0.036573056131601334,...","[-0.032860055565834045, -0.036573056131601334,..."
2,2020-05-29,KO,42.223438,42.684747,41.734991,42.223438,51125000.0,Learning from Warren Buffett's Decision to Buy...,Learning from Warren Buffett's Decision to Buy...,0.006597,0.004691,0.025470,0.025877,0.047347,-0.014468,-0.022314,"[-0.007852538488805294, -0.015758998692035675,...","[-0.007852538488805294, -0.015758998692035675,..."
3,2020-05-29,V,190.180375,192.370633,188.987469,190.903946,10211900.0,8 Stocks Viking Global Investors Continues to Buy,8 Stocks Viking Global Investors Continues to Buy,-0.004579,0.005704,0.008279,-0.008263,0.019289,-0.015500,-0.032907,"[-0.02816659025847912, -0.03834887966513634, 0...","[-0.02816659025847912, -0.03834887966513634, 0..."
4,2020-05-29,AAPL,78.254387,78.720112,77.572955,77.933281,153532400.0,DXC Technology (DXC) Q4 Earnings Top Estimates...,DXC Technology (DXC) Q4 Earnings Top Estimates...,0.012148,0.016701,0.022084,0.013589,0.075729,0.061570,0.167500,"[-0.015577776357531548, -0.005795145407319069,...","[-0.013864366337656975, -0.014639061875641346,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8790,2015-06-02,HLT,56.787502,57.099843,56.631330,56.982716,2869981.0,Hilton Worldwide building out Curio brand,Hilton Worldwide building out Curio brand,0.003414,0.002733,0.006805,-0.014951,-0.013542,-0.041384,-0.037314,"[-0.003722486784681678, 0.005681690294295549, ...","[-0.003722486784681678, 0.005681690294295549, ..."
8791,2015-06-01,HLT,56.807024,57.216972,56.611811,56.943672,3460136.0,Timeshares Trend: How Marriott Vacations Revam...,Timeshares Trend: How Marriott Vacations Revam...,0.000685,0.004097,0.003416,0.007485,-0.017440,-0.026390,-0.027837,"[0.006758211646229029, -0.001204228145070374, ...","[-0.008595649152994156, -0.0059041185304522514..."
8792,2015-06-01,MAR,72.173422,72.681038,71.398161,71.961151,1928200.0,Fitch ratings weighs in on lodging sector,Fitch ratings weighs in on lodging sector - Ti...,0.005865,0.017762,0.006498,0.005231,-0.009582,-0.012335,-0.009190,"[0.008254693821072578, -4.126824478589697e-06,...","[0.019749974831938744, 0.00014178613491822034,..."
8793,2015-06-01,CL,54.954373,55.292126,54.666051,55.110893,3447100.0,"Dividend, Yield, And Price: Inseparably Linked","Dividend, Yield, And Price: Inseparably Linked",-0.002548,-0.004957,-0.014866,-0.021686,0.002535,-0.023875,0.005500,"[-0.02472446858882904, -0.021279681473970413, ...","[-0.02472446858882904, -0.021279681473970413, ..."


In [4]:
news_df['1_day_return_positive'] = news_df['1_day_return'].apply(lambda x: 1 if x > 0 else 0)
news_df['2_day_return_positive'] = news_df['2_day_return'].apply(lambda x: 1 if x > 0 else 0)
news_df['3_day_return_positive'] = news_df['3_day_return'].apply(lambda x: 1 if x > 0 else 0)
news_df['4_day_return_positive'] = news_df['4_day_return'].apply(lambda x: 1 if x > 0 else 0)
news_df['7_day_return_positive'] = news_df['7_day_return'].apply(lambda x: 1 if x > 0 else 0)
news_df['10_day_return_positive'] = news_df['10_day_return'].apply(lambda x: 1 if x > 0 else 0)
news_df['30_day_return_positive'] = news_df['30_day_return'].apply(lambda x: 1 if x > 0 else 0)

In [5]:
# Splitting the data into train and test sets
test_size = int(len(news_df) * 0.1)
train_df = news_df[test_size:]
test_df = news_df[:test_size]

X_train = np.array([np.fromstring(record.strip('[]'), sep=',') for record in train_df['Concatenated_News_Embedding']])
X_test = np.array([np.fromstring(record.strip('[]'), sep=',') for record in test_df['Concatenated_News_Embedding']])

y_train = train_df['1_day_return_positive']
y_test = test_df['1_day_return_positive']

In [6]:
svm = SVC(kernel='linear', C=1.0, random_state=42)

# Train the classifier on the training data
svm.fit(X_train, y_train)

# Predict the labels of the test set
y_pred = svm.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)

Accuracy: 0.5085324232081911
Classification Report:
              precision    recall  f1-score   support

           0       0.60      0.16      0.25       456
           1       0.49      0.89      0.64       423

    accuracy                           0.51       879
   macro avg       0.55      0.52      0.44       879
weighted avg       0.55      0.51      0.43       879



In [7]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier on the training data
rf.fit(X_train, y_train)

# Predict the labels of the test set
y_pred = rf.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)

Accuracy: 0.5233219567690558
Classification Report:
              precision    recall  f1-score   support

           0       0.57      0.35      0.43       456
           1       0.50      0.71      0.59       423

    accuracy                           0.52       879
   macro avg       0.53      0.53      0.51       879
weighted avg       0.54      0.52      0.51       879



In [8]:
xgb = XGBClassifier(n_estimators=100, use_label_encoder=False, eval_metric='logloss', random_state=42)

# Train the classifier on the training data
xgb.fit(X_train, y_train)

# Predict the labels of the test set
y_pred = xgb.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [18:16:01] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy: 0.5199089874857793
Classification Report:
              precision    recall  f1-score   support

           0       0.55      0.45      0.49       456
           1       0.50      0.60      0.55       423

    accuracy                           0.52       879
   macro avg       0.52      0.52      0.52       879
weighted avg       0.52      0.52      0.52       879



In [9]:
# Build a feed-forward neural network
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=8, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy}')

# Predict the labels of the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Print classification report
from sklearn.metrics import classification_report
print('Classification Report:')
print(classification_report(y_test_classes, y_pred_classes))

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50


In [ ]:
# Build an LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(32, activation='relu'))
model.add(Dense(y_categorical.shape[1], activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=8, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy}')

# Predict the labels of the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Print classification report
from sklearn.metrics import classification_report
print('Classification Report:')
print(classification_report(y_test_classes, y_pred_classes))